In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from collections import deque
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

64-bit Processor,<br>
32 registers and each register is 64-bit<br>

Functional Units<br>
Integer Addition/Subtraction (IADD Rdst, Rsrc1, Rsrc2)  -  6 CC<br>
Integer Multiplication (IMUL Rdst, Rsrc1, Rsrc2)  -  12 CC<br>
Floating point Addition/Subtraction (FADD Rdst, Rsrc1, Rsrc2)  -  18 CC<br>
Floating point Multiplication (FADD Rdst, Rsrc1, Rsrc2)  -  30 CC<br>
Load (LD Rdst, Mem) - 1 CC<br>
Store (ST Mem, Rsrc) - 1 CC<br>
Logic Unit (AND/OR/XOR Rdst, Rsrc1, Rsrc2)  -  1 CC<br>
No Operation (NOP) 1CC<br>

In [49]:
import pandas as pd
from collections import deque

class Tomasulo:
    def __init__(self):
        self.clock = 0
        r_list = [f"r{i}" for i in range(32)]
        self.register_file = pd.DataFrame({r_list[i]: [[], -1] for i in range(32)}) 

        self.fus = {
            'iadd': {1, 2, 3}, 'imul': {1, 2}, 'fadd': {1, 2, 3},
            'fmul': {1, 2}, 'load': {1, 2, 3}, 'store': {1}, 'logic': {1, 2}
        }
        self.clock_cycles = {'iadd': 6, 'imul': 12, 'fadd': 18, 'fmul': 30, 'load': 1, 'store': 1, 'logic': 1}
        self.instruction_queue = deque()
        self.common_data_bus = []

        # Reservation Stations now have multiple entries for each FU
        self.reservation_stations = {
            f"{op}_{fu}": [0, 0, 0, 0, 0, 0] for op in self.fus for fu in self.fus[op]
        }

        self.num_reservation_stations = {
            f"{op}_{fu}": 1 for op in self.fus for fu in self.fus[op]
        }

        self.memory = {x: 5 for x in range(65536)}
        self.instruction_table = pd.DataFrame(columns=['cc', 'Instr', 'Issue', 'Execute', 'Write Result'])

    def check_status(self):
        print("Reservation Stations:")
        for key, value in self.reservation_stations.items():
            print(f"{key}: {value}")

        print("\nInstruction Table:")
        print(self.instruction_table)

    def execute(self, instruction, decode_string):
        result = 0
        op_type = instruction[0]

        # Find a free FU
        for fu_id in self.fus[op_type]:
            res_key = f"{op_type}_{fu_id}"
            if self.reservation_stations[res_key][5] == 'busy':
                self.clock += self.clock_cycles[op_type]

                if decode_string == 'fp':
                    result = self.compute_result(op_type, self.register_file[instruction[1]][1], self.register_file[instruction[2]][1])
                elif decode_string == 'load':
                    self.reservation_stations[res_key][4] += self.reservation_stations[res_key][2]
                    self.reservation_stations[res_key][4] = self.memory[self.reservation_stations[res_key][4]]

                self.instruction_table.loc[len(self.instruction_table)] = [
                    self.clock, instruction[0], 'done', 'done', ''
                ]
                
                self.write_result(instruction, result, decode_string, res_key)
                break

    def instruction_fetch(self, instructions):
        for row in instructions.itertuples(index=False):
            self.instruction_queue.append(list(row))

        while self.instruction_queue:
            issued_instructions = []
            for _ in range(len(self.instruction_queue)):
                temp_lst = self.instruction_queue.popleft()
                if self.issue(temp_lst):
                    issued_instructions.append(temp_lst)
                    self.instruction_table.loc[len(self.instruction_table)] = [
                        self.clock, temp_lst[0], 'done', '', ''
                    ]

            self.clock += 1  # Move to next cycle

            for instr in issued_instructions:
                self.execute(instr, self.decode(instr))

    def decode(self, instruction):
        return 'load' if instruction[0] == 'load' else ('store' if instruction[0] == 'store' else 'fp')

    def issue(self, instruction):
        decode_string = self.decode(instruction)
        op_type = instruction[0]

        for fu_id in self.fus[op_type]:
            res_key = f"{op_type}_{fu_id}"
            if self.num_reservation_stations[res_key] > 0:
                if decode_string == 'fp':
                    self._assign_fp_operands(instruction, res_key)
                else:
                    self._assign_mem_operands(instruction, res_key, decode_string)

                self.num_reservation_stations[res_key] -= 1
                self.check_status()
                return True

        self.instruction_queue.append(instruction)  
        return False 

    def _assign_fp_operands(self, instruction, res_key):
        """Assign floating point operation operands to reservation station."""
        if self.register_file[instruction[2]][0] != []:
            self.reservation_stations[res_key][0] = self.register_file[instruction[2]][0][1]
        else:
            self.reservation_stations[res_key][2] = int(self.register_file[instruction[2]][1])
            self.reservation_stations[res_key][0] = 0

        if self.register_file[instruction[3]][0] != []:
            self.reservation_stations[res_key][1] = self.register_file[instruction[3]][0][1]
        else:
            self.reservation_stations[res_key][3] = int(self.register_file[instruction[3]][1])
            self.reservation_stations[res_key][1] = 0

        self.reservation_stations[res_key][5] = 'busy'
        self.register_file[instruction[1]][0] = [instruction[0], self.num_reservation_stations[res_key]]

    def _assign_mem_operands(self, instruction, res_key, decode_string):
        """Assign memory operands to reservation station."""
        if self.register_file[instruction[3]][0] != []:
            self.reservation_stations[res_key][0] = self.register_file[instruction[3]][0][1]
        else:
            self.reservation_stations[res_key][2] = self.register_file[instruction[3]][1]
        
        self.reservation_stations[res_key][4] = int(instruction[2])
        self.reservation_stations[res_key][5] = 'busy'
        if decode_string == 'load':
            self.register_file[instruction[1]][0] = [instruction[0], self.num_reservation_stations[res_key]]

    def compute_result(self, operation, value1, value2):
        value1, value2 = int(value1), int(value2)
        return value1 + value2 if operation in ['iadd', 'fadd'] else value1 * value2

    def write_result(self, instruction, result, decode_string, res_key):
        """Write results back to registers or memory."""
        if decode_string in ['fp', 'load']:
            for key in self.register_file.columns:
                if self.register_file[key][0] and self.register_file[key][0][0] == decode_string:
                    self.register_file[key][1] = result
                    self.register_file[key][0] = []
            
            self.reservation_stations[res_key][2] = result
            self.reservation_stations[res_key][0] = 0
            self.reservation_stations[res_key][3] = result
            self.reservation_stations[res_key][1] = 0
        else:
            self.memory[self.reservation_stations[res_key][4]] = self.reservation_stations[res_key][3]
        
        self.reservation_stations[res_key][5] = ''
        self.num_reservation_stations[res_key] += 1
        self.instruction_table.loc[len(self.instruction_table)] = [
            self.clock, instruction[0], 'done', 'done', 'done'
        ]
        self.check_status()


In [50]:
def test_program(program):
    print("Code segment:")
    for row in program.itertuples(index = True):
        print("Line:",row[0],"\t",row[1:])

In [51]:
def run(program):
    toma = Tomasulo()
    toma.register_file['r10'][1] = 20
    toma.register_file['r11'][1] = 30
    toma.instruction_fetch(program)
    return 0

In [53]:
if __name__ == '__main__':
    '''TOMASULO IMPLEMENTATION'''
    '''PROCESSOR DETAILS
        16 bit processor (32 16-bit Registers)

        Number of clock cycles :
            Integer Addtion: 6 cc
            Integer Multiplication: 12 cc
            Floating Point Addition: 18 cc
            Floating Point Multiplication: 30 cc
            Load unit: 1 cc
            Store unit: 1 cc
            Logic unit: 1cc

        Has multiple reservation stations per Functional unit that enables register renaming and thus prevents WAR and WAW hazard.
    '''

    #Read the Program
    program = pd.read_csv('program1.txt', header = None)
    #print the Program
    test_program(program)

    #Run the Program using Tomasulo Processor
    exit_status = run(program)

    print(f"The program has been executed successfully with code {exit_status}.")

Code segment:
Line: 0 	 ('load', 'r1', '5', 'r10')
Line: 1 	 ('load', 'r2', '10', 'r11')
Line: 2 	 ('iadd', 'r3', 'r1', 'r2')
Line: 3 	 ('imul', 'r4', 'r3', 'r2')
Line: 4 	 ('store', 'r4', '0', 'r12')
Line: 5 	 ('load', 'r5', '15', 'r15')
Line: 6 	 ('iadd', 'r3', 'r5', 'r1')
Line: 7 	 ('imul', 'r6', 'r3', 'r2')
Line: 8 	 ('store', 'r6', '4', 'r16')
Line: 9 	 ('load', 'r1', '20', 'r13')
Line: 10 	 ('iadd', 'r7', 'r1', 'r4')
Reservation Stations:
iadd_1: [0, 0, 0, 0, 0, 0]
iadd_2: [0, 0, 0, 0, 0, 0]
iadd_3: [0, 0, 0, 0, 0, 0]
imul_1: [0, 0, 0, 0, 0, 0]
imul_2: [0, 0, 0, 0, 0, 0]
fadd_1: [0, 0, 0, 0, 0, 0]
fadd_2: [0, 0, 0, 0, 0, 0]
fadd_3: [0, 0, 0, 0, 0, 0]
fmul_1: [0, 0, 0, 0, 0, 0]
fmul_2: [0, 0, 0, 0, 0, 0]
load_1: [0, 0, 20, 0, 5, 'busy']
load_2: [0, 0, 0, 0, 0, 0]
load_3: [0, 0, 0, 0, 0, 0]
store_1: [0, 0, 0, 0, 0, 0]
logic_1: [0, 0, 0, 0, 0, 0]
logic_2: [0, 0, 0, 0, 0, 0]

Instruction Table:
Empty DataFrame
Columns: [cc, Instr, Issue, Execute, Write Result]
Index: []
Reservation S